<a href="https://colab.research.google.com/github/Esther-Wagatwe/EASY-ML/blob/master/Retrieval_Augmented_Generation_(RAG).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install required libraries
!pip install -q transformers sentence-transformers faiss-cpu openai
!pip install -q pdfplumber python-docx pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# STEP 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# STEP 3: Import libraries
import faiss
import numpy as np
import pdfplumber
import pandas as pd
from docx import Document
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration
import os


In [ ]:
# STEP 4: Set folder path in Google Drive
folder_path = "/content/drive/MyDrive/RAG"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# STEP 5: Define file extractor
def extract_text(file_path, ext):
    try:
        if ext == "pdf":
            text = ""
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
            return text
        elif ext == "docx":
            doc = Document(file_path)
            return "\n".join([p.text for p in doc.paragraphs])
        elif ext == "csv":
            df = pd.read_csv(file_path)
            return df.to_string(index=False)
        else:
            return ""
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return ""

In [ ]:
# STEP 6: Read all supported files in folder
supported_exts = ["pdf", "docx", "csv"]
raw_text = ""

for file in os.listdir(folder_path):
    full_path = os.path.join(folder_path, file)
    ext = file.split(".")[-1].lower()
    if ext in supported_exts:
        print(f"Reading: {file}")
        text = extract_text(full_path, ext)
        raw_text += text + "\n"

# Now raw_text contains the combined text from all files
print(f"\n Total characters extracted: {len(raw_text)}")

Reading: un_brand_report_web_august_2020_v36928.pdf
Reading: COVID-19-Pandemic-Impact-on-Kenyan-Education-Sector-Learner-Challenges-and-Mitigations.pdf
Reading: s12916-022-02367-4.pdf
Reading: Africa-CDC-Epidemic-Intelligence-Weekly-Report_3-February-2025 (1).pdf
Reading: WHO AFRO COVID_19_RESPONSE QUARTERLY Bulletin..pdf
Reading: African Development Review - 2021 - Anyanwu - The impact of COVID‐19 on African economies  An introduction.pdf

 Total characters extracted: 369566


In [ ]:
# STEP 7: Split into chunks
def chunk_text(text, max_chars=500):
    paragraphs = text.split("\n")
    chunks, current_chunk = [], ""
    for para in paragraphs:
        if len(current_chunk) + len(para) <= max_chars:
            current_chunk += para + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = para + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return [chunk for chunk in chunks if len(chunk) > 50]

doc_chunks = chunk_text(raw_text)

In [ ]:
# STEP 7: Embed chunks using SentenceTransformers
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(doc_chunks, convert_to_tensor=False)
doc_embeddings = np.vstack(doc_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# STEP 8: Build FAISS (Facebook AI Similarity Search) index
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)

In [ ]:
# STEP 9: Set your OpenAI API key


In [ ]:
# Updated: RAG using local T5 model (no OpenAI)
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
generator = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

def rag_local_t5_answer(question, top_k=3):
    # Embed query and retrieve
    query_embedding = embedder.encode([question])
    D, I = index.search(np.array(query_embedding), top_k)

    # Combine top passages
    context = "\n".join([doc_chunks[i] for i in I[0]])
    input_prompt = f"""
    Based on the following context, provide a comprehensive and detailed answer to the question.
    Elaborate on the key points found in the context and explain them thoroughly in two well-written paragraphs.

    Context:
    {context}

    Question: {question}
    """

    # Tokenize and generate
    input_ids = tokenizer.encode(input_prompt, return_tensors="pt", truncation=True, max_length=512)
    output_ids = generator.generate(input_ids, max_length=400, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# STEP 11: Ask a Question using local T5 model
question = "What is covid 19?"
answer = rag_local_t5_answer(question)

print("Question:", question)
print("\n Answer:\n", answer)


Question: What is covid 19?

 Answer:
 Covid-19 is a viral disease.
